In [6]:
import os
import requests
##Create service accounts and tokens
#nucleus_uri = os.environ['DOMINO_API_HOST']
api_proxy = os.environ['DOMINO_API_PROXY']
svc_accounts_endpoint= f'{api_proxy}/v4/serviceAccounts'
#token = requests.get(f'{api_proxy}/access_token').text
print(svc_accounts_endpoint)
resp = requests.get(svc_accounts_endpoint)
print(resp)
users=[]
if resp.status_code==200:
    for u in resp.json():
        users.append(u['username'])
print(users)




    

http://localhost:8899/v4/serviceAccounts
<Response [200]>
['svc-user-ds-1', 'svc-user-ds-2', 'svc-user-ds-3', 'svc-test-app-1', 'svc-prod-app-1', 'svc-admin']


In [4]:
svc_accounts=['svc-user-ds-1','svc-user-ds-2','svc-user-ds-3','svc-test-app-1','svc-prod-app-1','svc-admin']


In [7]:
for s in svc_accounts:
    if s not in users:
        data = {'username':s,'email':f'{s}@xyz.com'}
        resp = requests.post(svc_accounts_endpoint,json=data,headers={'Content-Type':'application/json'})
        print(f'Response for creating svc account {s} is {resp.status_code}')
        #Create a service account'{"username": "demo-sa", "email": "demo-sa@customer.com"}' 

In [8]:
#Now get Idp Id of all emails
import requests
resp = requests.get(svc_accounts_endpoint)
print(resp)
idp_by_user={}
if resp.status_code==200:
    for u in resp.json():
        idp_by_user[u['username']]=u['idpId']
print(idp_by_user)        

<Response [200]>
{'svc-user-ds-1': '004934b3-3efb-420a-9059-27ae0eb4babf', 'svc-user-ds-2': 'ecb7718d-e643-46c5-9d53-e3a89f174b10', 'svc-user-ds-3': 'cb2546aa-1b3c-4dff-b303-a5bbd989c08b', 'svc-test-app-1': '38015a03-8e66-48c9-8082-54503c206bbd', 'svc-prod-app-1': '81723029-997a-44e8-a71e-867bf38c54d4', 'svc-admin': '2c39f627-c906-47d8-a7f4-75d7915bd26f'}


In [9]:
results = []
token_name='token-for-svc-account-3'
for user, idpid in idp_by_user.items():
    print(user)

svc-user-ds-1
svc-user-ds-2
svc-user-ds-3
svc-test-app-1
svc-prod-app-1
svc-admin


In [15]:
results = []
#token_name='token-for-svc-account-3'
for user, idpid in idp_by_user.items():
    token_endpoint= f'{api_proxy}/v4/serviceAccounts/{idpid}/tokens'    
    data = {} 
    data['name']=f'{user}-token-app'
    
    
    resp = requests.post(token_endpoint,json=data,headers={'Content-Type':'application/json'})
    print(resp.status_code)

    if resp.status_code==201:
       out = resp.json()    
       results.append(out)

print(results)

  

201
201
201
201
201
201
[{'name': 'svc-user-ds-1-token-app', 'serviceAccountIdpId': '004934b3-3efb-420a-9059-27ae0eb4babf', 'token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJzdjJuOGdVTy11eEQ3V3cxOURPSGZxZW82TnowS1dubVMtdHFLakxpZzI0In0.eyJleHAiOjE3MzE0Mzg5NTIsImlhdCI6MTcyMTA3MDk1MiwianRpIjoiNGQzNTg0ZGMtNjdkOS00NTJmLWE3NTgtNGNiMmY4ZTExODcwIiwiaXNzIjoiaHR0cHM6Ly9zZWN1cmVkczUzNzk5LmNzLmRvbWluby50ZWNoL2F1dGgvcmVhbG1zL0RvbWlub1JlYWxtIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6IjAwNDkzNGIzLTNlZmItNDIwYS05MDU5LTI3YWUwZWI0YmFiZiIsInR5cCI6IkJlYXJlciIsImF6cCI6ImRvbWluby1zZXJ2aWNlLWFjY291bnRzIiwic2Vzc2lvbl9zdGF0ZSI6ImI1NmE5ZDlhLTA3NDUtNGU0NS1iYWJkLTE3YzQ3YWRkMmQ0YSIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1kb21pbm9yZWFsbSJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImRvbWluby1zZXJ2aWNlLWFjY291bnRzIjp7InJvbGVzIjpbImRvbWluby1zZXJ2aWNlLWFjY291bnQiXX0sImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sIn

In [16]:
#print(results)
file_path = "/mnt/sa-tokens.json"
import json
# Write the JSON object to a file
with open(file_path, 'w') as file:
    json.dump(results, file, indent=4)  